In [ ]:
import sys
from nplinker.nplinker import NPLinker
%reload_ext autoreload
%autoreload 2

In [ ]:
npl = NPLinker('latest_api_demo.toml')
if not npl.load_data():
    raise Exception('Failed to load data')

## Rosetta-stone linking (nplinker version)

In [ ]:
from nplinker.scoring.rosetta import rosetta
ro = rosetta.Rosetta(npl.data_dir, npl.root_dir, npl.dataset_id,ignore_genomic_cache = True)
rhits = ro.run(npl.spectra, npl.bgcs)
print('Rosetta hits: {}'.format(len(rhits)))

In [ ]:
for r in rhits:
    print(r)

In [ ]:
k = list(ro.bgc_hits.keys())[0]
summary_scores = ro.generate_bgc_summary_scores()
for key,item_list in summary_scores.items():
    for mib,score in item_list.items():
        print(key,'\t',mib,score)

In [ ]:
kk = list(summary_scores.keys())
temp = ro.bgc_hits[kk[-2]]['BGC0000644']
tt = temp['individual_hits']
print(tt)
print(temp['all_mibig_genes'])
print(temp['all_bgc_genes'])

In [ ]:
import csv
spec_hits = ro.spec_hits
# Write this out as a .tsv file to test the DB loading
with open('carnegie_rosetta_hits.tsv','w') as f:
    writer = csv.writer(f,delimiter='\t')
    heads = ['#Scan#','GNPS_ID','Score']
    writer.writerow(heads)
    for spec,hits in spec_hits.items():
        for hit in hits:
            writer.writerow([spec.spectrum_id, hit[0], hit[1]])

## Todo:

- At the moment we get lots of hits per GNPS,MiBIG pair because they are in lots of BGCs
- We also should percolate the scores (both of the spectral match and the knownclusterblast) to the output
- Parameterise (at least) two parameters in the spectral matching: score threshold and ms1_tol. At the moment, MS1_tol will only find things with near identical MS1 m/z, which precludes analogues.
- The code for getting the knownclusterblast name and parsing the knownclusterblast file is horrific... :-)